In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# load our serialized face detector model from disk
# initialise the pretrained face detector model provided by OpenCV
prototxtPath = "deploy.prototxt"
weightsPath = "res10_300x300_ssd_iter_140000.caffemodel"

# loading the pretrained face detector model using readNet() into the memory
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)


# load the face mask detector model created previously into the memory
maskNet = load_model("mask_detector.model")
print(maskNet)

In [3]:
image = cv2.imread('examples/image03.jpg')
orig = image.copy()
(h, w) = image.shape[:2]
print(image.shape,h,w)

(360, 540, 3) 360 540


In [4]:
# construct a blob from the image
blob=cv2.dnn.blobFromImage(image, 1.0, (300, 300),(104.0, 177.0, 123.0))
print(blob)

# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
faceNet.setInput(blob)
detections = faceNet.forward()
print(detections)

[[[[117. 117. 117. ... 130. 130. 130.]
   [117. 117. 117. ... 130. 130. 130.]
   [117. 117. 117. ... 130. 130. 130.]
   ...
   [ 26.  32.  32. ... 120. 120. 120.]
   [ 26.  33.  33. ... 120. 120. 120.]
   [ 26.  32.  34. ... 120. 120. 120.]]

  [[ 45.  45.  45. ...  58.  58.  58.]
   [ 45.  45.  45. ...  58.  58.  58.]
   [ 45.  45.  45. ...  58.  58.  58.]
   ...
   [-20. -14. -14. ...  51.  51.  51.]
   [-20. -13. -12. ...  51.  51.  51.]
   [-20. -14. -12. ...  51.  51.  51.]]

  [[103. 103. 103. ... 114. 114. 114.]
   [103. 103. 103. ... 114. 114. 114.]
   [103. 103. 103. ... 114. 114. 114.]
   ...
   [ 63.  69.  69. ... 108. 108. 108.]
   [ 63.  70.  70. ... 108. 108. 108.]
   [ 63.  69.  71. ... 108. 108. 108.]]]]
[INFO] computing face detections...
[[[[ 0.          1.          0.9984889   0.2934639   0.18077049
     0.51950324  0.54103553]
   [ 0.          1.          0.12556168  4.1601286   4.001864
     4.8384743   4.984943  ]
   [ 0.          1.          0.11896139  0.1623297

In [ ]:
# loop over the detections
for i in range(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with
	# the detection
	confidence = detections[0, 0, i, 2]
	#print(confidence)
    
	# filter out weak detections by ensuring the confidence is greater than the minimum confidence
	if confidence > 0.5:
	# compute the (x, y)-coordinates of the bounding box for the object
		print(detections[0, 0, i, 3:7])   

		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		print(box)

			# ensure the bounding boxes fall within the dimensions of the frame
		(startX, startY) = (max(0, startX), max(0, startY))
		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))


			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
		face = image[startY:endY, startX:endX]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
		face = cv2.resize(face, (224, 224))
		face = img_to_array(face)
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)


		# pass the face through the model to determine if the face has a mask or not
		(mask, withoutMask) = maskNet.predict(face)[0]


		# determine the class label and color we'll use to draw the bounding box and text
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)


		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
		# display the label and bounding box rectangle on the output frame
		cv2.putText(image, label, (startX, startY - 10),
		cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)


# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)

[0.2934639  0.18077049 0.51950324 0.54103553]
[158.47049832  65.07737517 280.53174734 194.77279186]
